In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Polycystic_Ovary_Syndrome"
cohort = "GSE87435"

# Input paths
in_trait_dir = "../../input/GEO/Polycystic_Ovary_Syndrome"
in_cohort_dir = "../../input/GEO/Polycystic_Ovary_Syndrome/GSE87435"

# Output paths
out_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/GSE87435.csv"
out_gene_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/gene_data/GSE87435.csv"
out_clinical_data_file = "../../output/preprocess/Polycystic_Ovary_Syndrome/clinical_data/GSE87435.csv"
json_path = "../../output/preprocess/Polycystic_Ovary_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Microarray expression data derived  from ovaries of women with Polycystic Ovary Syndrome and Female to Male Transexual individuals after treatment with Testosterone"
!Series_summary	"This microarray analysis identified differentially regulated expression of 326 genes (p value=<0.05 and fold change of +/-2). Based on these genes, a biological LH network was generated consisting molecules where the significance of the association was measured by Fisher's exact test."
!Series_summary	"We used microarray gene expression analysis to dissect the LH-induced gene expression in the ovaries of PCOS women."
!Series_overall_design	"RNA was isolated from ovary biopsies of PCOS women and TSX individuals to investigate LH induced gene expression."
Sample Characteristics Dictionary:
{0: ['gender: female', 'gender history: female to male transsexual'], 1: ['disease state: PCOS', 'tissue: ovary'], 2: ['tissue: ovary', nan]}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series title and summary, this dataset seems to contain microarray gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (PCOS)
# Key 1 contains 'disease state: PCOS' - this should be our trait row
trait_row = 1  

# For age
# There's no mention of age in the sample characteristics
age_row = None

# For gender
# Key 0 contains gender information
gender_row = 0

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert PCOS trait to binary (0 = control, 1 = PCOS)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'PCOS' in value:
        return 1
    else:
        return 0

def convert_gender(value):
    """Convert gender to binary (0 = female, 1 = male)
    Note: In this context, we're dealing with biological sex rather than gender identity
    Female-to-male transsexuals are biologically female (0)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Female is 0, Male is 1
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower() and 'female to male' not in value.lower():
        return 1
    else:
        return 0  # Female to male transsexuals are recorded as female (0) biologically

# 3. Save Metadata - Initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Load clinical data from previous step (assume it's available)
        clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"))
        
        # Extract clinical features
        selected_clinical = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=None  # No age data available
        )
        
        # Preview the selected clinical features
        preview = preview_df(selected_clinical)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical feature extraction: {e}")
        # If clinical_data.csv doesn't exist, we'll get an error, but that's expected in some cases


Error in clinical feature extraction: [Errno 2] No such file or directory: '../../input/GEO/Polycystic_Ovary_Syndrome/GSE87435/clinical_data.csv'


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Polycystic_Ovary_Syndrome/GSE87435/GSE87435-GPL96_series_matrix.txt.gz
Gene data shape: (22283, 18)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers
# The identifiers start with numbers followed by "_at", "_s_at", "_x_at" pattern
# These are Affymetrix probe IDs from the GPL97 platform (HG-U133 Plus 2.0), not standard gene symbols
# They need to be mapped to official gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns for gene IDs and gene symbols in the gene annotation dataframe
# Based on the preview, the ID column contains the same probe identifiers as in the gene expression data
# The Gene Symbol column contains the gene symbols we need to map to
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get a gene mapping dataframe using the helper function
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Check the mapping
print("\nSample of gene mapping:")
print(gene_mapping.head())
print(f"Number of mappings found: {len(gene_mapping)}")

# 3. Convert probe-level measurements to gene expression data by applying the gene mapping
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Check the resulting gene expression data
print("\nGene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print("First 5 gene symbols:")
print(gene_data.index[:5])

# Normalize gene symbols to handle synonyms and aggregate rows with the same normalized symbol
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nGene expression data after normalization:")
print(f"Shape after normalization: {gene_data.shape}")
print("First 5 normalized gene symbols:")
print(gene_data.index[:5])



Sample of gene mapping:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A
Number of mappings found: 38249

Gene expression data after mapping:
Shape: (13830, 18)
First 5 gene symbols:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS'], dtype='object', name='Gene')

Gene expression data after normalization:
Shape after normalization: (13542, 18)
First 5 normalized gene symbols:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. First, extract and save the clinical data since it's missing
# Get the SOFT and matrix file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get the background info and clinical data again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define the conversion functions from Step 2
def convert_trait(value):
    """Convert PCOS trait to binary (0 = control, 1 = PCOS)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'PCOS' in value:
        return 1
    else:
        return 0

def convert_gender(value):
    """Convert gender to binary (0 = female, 1 = male)
    Note: In this context, we're dealing with biological sex rather than gender identity
    Female-to-male transsexuals are biologically female (0)"""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Female is 0, Male is 1
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower() and 'female to male' not in value.lower():
        return 1
    else:
        return 0  # Female to male transsexuals are recorded as female (0) biologically

# Extract clinical features with the correct row indices from previous steps
trait_row = 1  # Contains "disease state: PCOS"
gender_row = 0  # Contains gender information
age_row = None  # Age information is not available in this dataset

# Process and save clinical data
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=None,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to: {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 2. Normalize gene symbols using synonym information from NCBI
print("\nNormalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene identifiers:")
print(gene_data.index[:10])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
# First check how many samples have missing trait values
if trait in linked_data.columns:
    missing_trait = linked_data[trait].isna().sum()
    print(f"Samples with missing trait values: {missing_trait} out of {len(linked_data)}")

# Check gene missing value percentages
gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
gene_missing_pct = linked_data[gene_cols].isna().mean()
genes_to_keep = gene_missing_pct[gene_missing_pct <= 0.2].index
print(f"Genes with ≤20% missing values: {len(genes_to_keep)} out of {len(gene_cols)}")

# Check sample missing value percentages
if len(gene_cols) > 0:
    sample_missing_pct = linked_data[gene_cols].isna().mean(axis=1)
    samples_to_keep = sample_missing_pct[sample_missing_pct <= 0.05].index
    print(f"Samples with ≤5% missing gene values: {len(samples_to_keep)} out of {len(linked_data)}")

# Apply missing value handling
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
trait_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This dataset contains gene expression data from ovary biopsies of women with PCOS and female-to-male transsexual individuals, focusing on LH-induced gene expression."
is_gene_available = len(gene_data) > 0
is_trait_available = trait in linked_data.columns
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=trait_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable and linked_data_clean.shape[0] > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Clinical data saved to: ../../output/preprocess/Polycystic_Ovary_Syndrome/clinical_data/GSE87435.csv
Clinical data preview:
{'GSM2331292': [1.0, 0.0], 'GSM2331294': [1.0, 0.0], 'GSM2331296': [1.0, 0.0], 'GSM2331298': [1.0, 0.0], 'GSM2331300': [1.0, 0.0], 'GSM2331302': [1.0, 0.0], 'GSM2331304': [0.0, 0.0], 'GSM2331306': [0.0, 0.0], 'GSM2331308': [0.0, 0.0], 'GSM2331310': [0.0, 0.0], 'GSM2331312': [0.0, 0.0], 'GSM2331314': [0.0, 0.0], 'GSM2331316': [0.0, 0.0], 'GSM2331318': [0.0, 0.0], 'GSM2331320': [0.0, 0.0], 'GSM2331322': [0.0, 0.0], 'GSM2331324': [0.0, 0.0], 'GSM2331326': [0.0, 0.0]}

Normalizing gene symbols...


Gene data shape after normalization: (13542, 18)
First 10 normalized gene identifiers:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB',
       'AAK1', 'AAMDC'],
      dtype='object', name='Gene')
Normalized gene data saved to: ../../output/preprocess/Polycystic_Ovary_Syndrome/gene_data/GSE87435.csv

Linking clinical and genetic data...
Linked data shape: (18, 13544)
Linked data preview (first 5 rows, 5 columns):
            Polycystic_Ovary_Syndrome  Gender     A1CF      A2M   A4GALT
GSM2331292                        1.0     0.0  3.80567  10.1223  2.02299
GSM2331294                        1.0     0.0  3.19719  10.8777  2.09732
GSM2331296                        1.0     0.0  3.14119  10.6665  2.14744
GSM2331298                        1.0     0.0  3.00950  10.7288  2.21295
GSM2331300                        1.0     0.0  3.03581  11.3793  2.10235

Handling missing values...
Samples with missing trait values: 0 out of 18


Genes with ≤20% missing values: 13542 out of 13542
Samples with ≤5% missing gene values: 18 out of 18


Linked data shape after handling missing values: (18, 13544)

Checking for bias in dataset features...
For the feature 'Polycystic_Ovary_Syndrome', the least common label is '1.0' with 6 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Polycystic_Ovary_Syndrome' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 18 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Polycystic_Ovary_Syndrome/GSE87435.csv
